In [22]:
from pymongo import MongoClient
from sshtunnel import SSHTunnelForwarder
import pprint
from bson.objectid import ObjectId
import pyperclip as clip
import json
import tensorflow as tf
import pandas as pd
import numpy as np
import itertools as it
# from datasu import auc
# import pixiedust
# clip.copy(json.dumps(m1['msg']))
import nbimporter
from common_functions import *


### funcs

In [185]:
import matplotlib.pyplot as plt
from matplotlib import colors
def background_gradient(s, m=None, M=None, cmap='summer', low=0, high=0):
    if m is None:
        m = s.min().min()
    if M is None:
        M = s.max().max()
    rng = M - m
    norm = colors.Normalize(m ,M)
    normed = s.apply(lambda x: norm(x.values))
    cm = plt.cm.get_cmap(cmap)
    c = normed.applymap(lambda x: colors.rgb2hex(cm(x)))
    ret = c.applymap(lambda x: 'background-color: %s' % x)
    return ret

### load data

In [85]:
# MONGO_HOST = "ec2-52-23-187-115.compute-1.amazonaws.com"
MONGO_DB = "marketpulzz"
# server = SSHTunnelForwarder(
#     MONGO_HOST,
#     ssh_username='ubuntu',
#     ssh_pkey="/Users/sasha/.ssh/mmuze.pem",
# #     ssh_private_key_password="secret",
#     remote_bind_address=('127.0.0.1', 27017),
#     local_bind_address=('127.0.0.1', 63328),
#     set_keepalive = 5,
# )

# server.start()
# client = MongoClient('127.0.0.1', server.local_bind_port) # server.local_bind_port is assigned local port
# db = client[MONGO_DB]

In [86]:
client_local = MongoClient('127.0.0.1', 27017)
db_local = client_local[MONGO_DB]

In [87]:
"STS1-v2.0".replace('STS1-','')

'v2.0'

In [246]:
def get_bert_exp_data(exp):
#     emb_metadata = exp['emb_metadata']       
    fields = {
        'notebook': exp['notebook'].replace('STS1-',''),
        'num_intents': len(exp['data_metadata']['intents']),
        'msg_fields': str(exp.get('exp_metadata',{}).get('fields',{'fields': ['text']})),
        'msg_vector_desc': str(exp.get('exp_metadata',{}).get('msg_vector_desc',{})),
        'encoder': exp['encoder'],
        'model': exp['emb_metadata'].get('model','-'),
        'pooling_strategy': exp['emb_metadata'].get('pooling_strategy','-'),
        'pooling_layer': exp['emb_metadata'].get('pooling_layer','-'),
        'dimensions': exp['emb_metadata'].get('dimensions','-'),
        'metric': exp['metric']['name'],
        'value': exp['metric']['value']
        
    }
    return fields

In [263]:
results_cursor = db_local.sts1_bert_use.find({})
results = list(map(get_bert_exp_data,results_cursor))
df_results = pd.DataFrame(results)
df_results.head()

,dimensions,encoder,metric,model,msg_fields,msg_vector_desc,notebook,num_intents,pooling_layer,pooling_strategy,value
0,1024,bert,MAP_DQ,uncased_L-24_H-1024_A-16,['text'],{'text': 'emb(1)'},v1.0,13,-1-2,REDUCE_MEAN,0.341974
1,1024,bert,MAP_TP,uncased_L-24_H-1024_A-16,['text'],{'text': 'emb(1)'},v1.0,13,-1-2,REDUCE_MEAN,0.429642
2,1024,bert,AUC_DQ,uncased_L-24_H-1024_A-16,['text'],{'text': 'emb(1)'},v1.0,13,-1-2,REDUCE_MEAN,0.617663
3,1024,bert,AUC_TP,uncased_L-24_H-1024_A-16,['text'],{'text': 'emb(1)'},v1.0,13,-1-2,REDUCE_MEAN,0.573045
4,512,use,MAP_DQ,https://tfhub.dev/google/universal-sentence-en...,['text'],{'text': 'emb(1)'},v1.0,13,-,-,0.463968


In [264]:
df_sts1_v1_summary = df_results.pivot_table(index=['notebook','msg_vector_desc','num_intents','encoder','dimensions','pooling_strategy', 'pooling_layer',], 
                                            values=['value'],columns=['metric'])
# import seaborn as sns
# cm = sns.light_palette("yellow", as_cmap=True)
df_sts1_v1_summary.style.background_gradient(cmap='summer')\
                        .format("{:.2%}") \
                        .set_properties(**{'color': 'black', 'font-weight': 'bold'})

### all data

In [265]:
df_sts1_best_pooling_strg = df_sts1_v1_summary[np.in1d(df_sts1_v1_summary.index.get_level_values(5), ['REDUCE_MEAN', '-']) &
                                               np.invert(np.in1d(df_sts1_v1_summary.index.get_level_values(0), ['v2.4', 'v2.5', 'v2.6']))]

df_sts1_best_pooling_strg.style.background_gradient(cmap='summer') \
                          .format("{:.2%}") \
                          .set_properties(**{'color': 'black', 'font-weight': 'bold'})

### compare best msg representations - with entities vs without entites - all data

In [267]:
df_sts1_v1_v2_best_compare = df_sts1_v1_summary[np.in1d(df_sts1_v1_summary.index.get_level_values(0), ['v2.0', 'v2.7', 'v2.8']) & 
                                                np.in1d(df_sts1_v1_summary.index.get_level_values(3), ['use'])]

df_sts1_v1_v2_best_compare.style.background_gradient(cmap='summer') \
                          .format("{:.2%}") \
                          .set_properties(**{'color': 'black', 'font-weight': 'bold'})

### compare best msg representations - with entities vs without entites - data only with entities

In [227]:
df_sts1_v1_v2_msgrepr_compare = df_sts1_v1_summary[np.in1d(df_sts1_v1_summary.index.get_level_values(0), ['v2.4', 'v2.5', ])]

# df_sts1_v1_v2_msgrepr_compare.style.apply(background_gradient, axis=None) \
#                           .format("{:.2%}") \
#                           .set_properties(**{'color': 'black', 'font-weight': 'bold'})

df_sts1_v1_v2_msgrepr_compare.style.background_gradient(cmap='summer') \
                             .set_properties(**{'color': 'black', 'font-weight': 'bold'})

### USE vs BERT - data wihout entities

In [268]:
df_sts1_v1_v2_data_wihout_entities_compare = df_sts1_v1_summary[np.in1d(df_sts1_v1_summary.index.get_level_values(0), ['v2.6'])]

df_sts1_v1_v2_data_wihout_entities_compare.style.background_gradient(cmap='summer') \
                          .format("{:.2%}") \
                          .set_properties(**{'color': 'black', 'font-weight': 'bold'})

In [270]:
df_sts1_best_pooling_strg = df_sts1_v1_summary[np.in1d(df_sts1_v1_summary.index.get_level_values(5), ['REDUCE_MEAN', '-']) &
                                               np.invert(np.in1d(df_sts1_v1_summary.index.get_level_values(0), ['v1.0']))]

df_sts1_best_pooling_strg.style.background_gradient(cmap='summer') \
                          .format("{:.2%}") \
                          .set_properties(**{'color': 'black', 'font-weight': 'bold'})